# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install gradio==6.1.0 openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. 流式输出

## 2.1 流式输出简介

在传统 LLM 应用中，我们常见的交互模式是：发送完整请求 → 模型思考 → 等待一段时间 → 返回完整答案。

但在真实应用（尤其是聊天、智能体、工具调用）中，这种“全等式响应”会造成以下问题：
- 响应延迟高（要等所有 token 输出完才显示）；
- 用户体验差（看起来卡顿）；
- 无法实时展示模型的推理过程或中间阶段（如 tool 调用、状态更新）。

所以为了解决这个问题，就出现了流式输出（Streaming）的机制，让模型还没有生成完答案的时候就能够获取正在输出的结果。目前大部分的大模型对话应用都已经实现了该技术！

## 2.2 普通输出与流式输出对比

### 2.2.1 普通输出

普通输出我们需要等待比较长时间才能获得回复：

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = PromptTemplate.from_template("请用一句话介绍一下 {topic}")

chain = prompt | llm 

result = chain.invoke({"topic": "人工智能"})
print(result)

content='人工智能是模拟人类智能，使机器具备学习、推理、感知与决策等能力以自主完成复杂任务的先进技术。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 5, 'total_tokens': 30, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-4.0-turbo-128k', 'system_fingerprint': None, 'id': 'as-w0w6h9wcqe', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b2fb2-bb50-7f93-87df-44634676cd1a-0' usage_metadata={'input_tokens': 5, 'output_tokens': 25, 'total_tokens': 30, 'input_token_details': {}, 'output_token_details': {}}


### 2.2.2 流式输出

流式输出我们可以看到一个个 token 输出展示出来：

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = PromptTemplate.from_template("请用一句话介绍一下 {topic}")

chain = prompt | llm 

for chunk in chain.stream({"topic": "人工智能"}):
  # chunk 是 ChatMessage 对象或字符串片段
  print(chunk.content, end="", flush=True)

人工智能是模拟人类智能，使机器具备学习、推理、感知与决策等能力以完成复杂任务的跨学科技术。

## 2.3 Gradio 页面实现流式输出

### 2.3.1 在 ChatBot 中实现流式输出

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os

llm = ChatOpenAI(
model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  MessagesPlaceholder(variable_name="chat_history"), 
  ("human", "{input}")])

chain = prompt | llm

session_store = {}

def get_session_history(session_id):
  if session_id not in session_store:
    session_store[session_id] = ChatMessageHistory()
  return session_store[session_id]

chain_with_history = RunnableWithMessageHistory(
  runnable=chain,
  get_session_history=get_session_history,
  input_messages_key="input",
  history_messages_key="chat_history"
)

import gradio as gr

session_id = "session_test5"

def chat_stream(message, history):
  history = history or []
  full_reply = ""

  # 使用 stream() 逐步输出
  for chunk in chain_with_history.stream(
    {"input": message},
    config={"configurable": {"session_id": session_id}}):
    delta = getattr(chunk, "content", None) or ""
    full_reply += delta

    # 逐步返回累积的完整内容
    yield "", history + [
      {"role": "user", "content": message},
      {"role": "assistant", "content": full_reply}]
    
with gr.Blocks() as demo:
  gr.Markdown("## 🤖 流式聊天机器人（带记忆功能）")
  chatbot = gr.Chatbot(label="LangChain ChatBot")
  msg = gr.Textbox(placeholder="和我聊聊吧...", lines=1)

  # 用 stream 函数绑定 msg.submit
  msg.submit(chat_stream, [msg, chatbot], [msg, chatbot])

  gr.ClearButton([msg, chatbot])

demo.launch()

c:\Users\76391\.conda\envs\chapter_3_2025_12_15\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


### 2.3.2 在 ChatInterface 中实现流式输出

In [6]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_store = {}

def get_session_history(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]


llm = ChatOpenAI(
    model="ernie-3.5-8k",
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

chain = prompt | llm

chain_with_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

def llm_response(message, history):
    session_id = "session_test6"
    stream = chain_with_history.stream(
        {"input": message},
        config={"configurable": {"session_id": session_id}})

    partial_text = ""
    
    for chunk in stream: 
        if hasattr(chunk, "content") and chunk.content:
            partial_text += chunk.content
            yield partial_text


demo = gr.ChatInterface(fn=llm_response)

demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


# 3. 输出解析器

## 3.1 简介

在传统的 LLM 应用中，模型的输出结果通常是纯自然语言文本，例如一大段描述性回答。

然而，在很多实际应用中（如问答系统、数据抽取、代码生成、智能体任务分解等），我们希望模型的输出能被程序直接解析和使用，而不仅仅是“读起来通顺”的文字。

为了解决这个问题，就出现了“结构化输出（Structured Output）”机制——让大模型在生成内容时，遵循特定的结构或格式（如 JSON、Pydantic、Schema），从而实现可控、可解析、可编程化的结果返回。

## 3.2 字符串解析器（StrOutputParser）

在前面 LCEL 里我们知道，在经过 llm 后，我们返回内容的格式是 AIMessage。但是这个格式是 LangChain 里自行定义的类，后处理起来比较麻烦。因此我们可以考虑在后面添加一个字符串解析器（StrOutputParser），从而使得返回的内容格式变为字符串，这样相对就好处理一些

### 3.2.1 未添加输出解析器

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = PromptTemplate.from_template("请用一句话介绍一下 {topic}")

chain = prompt | llm 

result = chain.invoke({"topic": "一句话介绍一下人工智能"})
print(type(result), result)

<class 'langchain_core.messages.ai.AIMessage'> content='人工智能是模拟人类智能，使机器具备学习、推理、感知与决策等能力的技术领域。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 7, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-4.0-turbo-128k', 'system_fingerprint': None, 'id': 'as-dfrmyevhgb', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b2fb2-dad7-7a01-8a39-aad1978ec462-0' usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {}, 'output_token_details': {}}


### 3.2.2 添加字符串解析器

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os

llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = PromptTemplate.from_template("请用一句话介绍一下 {topic}")

parser = StrOutputParser()

chain = prompt | llm | parser

result = chain.invoke({"topic": "一句话介绍一下人工智能"})
print(type(result), result)

<class 'str'> 人工智能是模拟人类智能，使机器能执行学习、推理、解决问题等复杂任务的先进技术。


## 3.3 列表解析器（CommaSeparatedListOutputParser）
当我们需要将输出的内容转化为一个格式化的列表，那我们就需要使用LangChain 中的输出解析器 CommaSeparatedListOutputParser。其原理是基于逗号将多个元素进行分隔（如地名、商品名、关键字等）。

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_openai import ChatOpenAI
import os

# 初始化模型
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
# 初始化解析器
parser = CommaSeparatedListOutputParser()

# 获取格式指令（后续载入到内部提示词中）
format_instructions = parser.get_format_instructions()
# 创建内部对话提示词模版
prompt = ChatPromptTemplate.from_template("""
提取以下句子中的编程语言，按格式要求输出：{format_instructions}
句子：{input}""")

chain = prompt | llm | parser
response = chain.invoke({
  "input": "他熟悉Python、Java和C++，还学过JavaScript",
  "format_instructions": format_instructions
})

print("列表结果：", response) # ['`Python', 'Java', 'C++', 'JavaScript`']

列表结果： ['`Python', 'Java', 'C++', 'JavaScript`']


## 3.4 字典解析器

## 3.4.1 StructuredOutputParser

StructuredOutputParser 会将模型输出的结构化字符串解析成一个 Python 字典（dict）。为了精确的更改对应的内容，需要通过 ResponseSchema 来对字典的键和值进行明确，也就是下面所示的 name 和 description 。 

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.output_parsers import StructuredOutputParser, ResponseSchema 
from langchain_openai import ChatOpenAI
import os


# 1. 定义字段 schema（名称+描述）
response_schemas = [
  ResponseSchema(name="product", description="产品名称"),
  ResponseSchema(name="price", description="产品价格（数字）"),
  ResponseSchema(name="rating", description="评分（1-5的数字）")
]
# 2. 初始化结构化解析器
parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 3. 格式指令与提示词
format_instructions = parser.get_format_instructions()

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
# 设置提示词模版
prompt = ChatPromptTemplate.from_template("""
解析以下产品评价中的信息，按格式要求输出：
{format_instructions}
评价：{input}
""")

chain = prompt | llm | parser
response = chain.invoke({
  "input": "这款无线耳机很好用，价格399元，我给4.5星好评",
  "format_instructions": format_instructions
})
print("结构化结果：", response)
print(type(response))

结构化结果： {'product': '无线耳机', 'price': '399', 'rating': '4.5'}
<class 'dict'>


### 3.4.2 JsonOutputParser + Pydantic

但是前面这个方法已经不再被推荐使用了（已经移动到 langchain_classic 中）

假如我们希望能够对键和值的数据类型进行控制，我们可以使用 JsonOutputParser + Pydantic 的方式进行实现，其会将 LLM 输出解析为 JSON 字符串 → 然后使用你定义的 Pydantic 模型 对这个结构进行严格类型校验与验证。

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# 使用 Pydantic 模型定义输出结构
class ProductInfo(BaseModel):
 product: str = Field(description="产品名称")
 price: float = Field(description="产品价格")
 rating: float = Field(description="评分（1-5的数字）")
parser = JsonOutputParser(pydantic_object=ProductInfo)
format_instructions = parser.get_format_instructions()

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = ChatPromptTemplate.from_template("""
解析以下产品评价中的信息，按格式要求输出：
{format_instructions}
评价：{input}""")

chain = prompt | llm | parser
response = chain.invoke({
  "input": "这款无线耳机很好用，价格399元，我给4.5星好评",
  "format_instructions": format_instructions
})
print("结构化结果：", response)
print(type(response))

结构化结果： {'product': '无线耳机', 'price': 399, 'rating': 4.5}
<class 'dict'>


## 3.5 格式化解析器

在更新后的 LangChain 格式化输出文档中，一种结合了模型和结构化输出的方法 llm.with_structured_output()，这个方法可以直接放在 prompt 后使用。

### 3.5.1 ChatOpenAI 格式化解析器

在 ChatOpenAI 模型里的 with_structured_output() 中支持输入三种格式的内容：
- function_calling：使用 OpenAI 工具调用的方法（但是当前的模型并不支持）
- json_mode：基于 OpenAI 的 JSON 模式，强制模型输出纯 JSON 字符串
- json_schema（默认使用）：基于 OpenAI 最新的结构化输出 API，直接通过 response_format 参数指定 JSON schema，模型会严格按照 schema 生成输出

![gr.ren](images/新6.png)

除此之外，Include raw 参数表示最后输出是否返回完整的字典信息（包含三个键 raw, parsed 和 parsing_error）而非单纯的结构化输出内容。

#### schema 类型
那在函数注释中，显示了可被传入的 schema 格式包括：

- Pydantic class

In [12]:
from pydantic import BaseModel, Field

class ProductInfo(BaseModel):
  """产品信息结构，包括名称、价格与评分"""
  product: str = Field(description="产品名称")
  price: float = Field(description="产品价格")
  rating: float = Field(description="评分（1-5的数字）")

- TypedDict class

In [13]:
from typing_extensions import TypedDict, Annotated

class ProductInfoDict(TypedDict):
  """产品信息结构，包括名称、价格与评分"""
  product: Annotated[str, {"description": "产品名称"}]
  price: Annotated[float, {"description": "产品价格（数字）"}]
  rating: Annotated[float, {"description": "评分（1-5）"}]

- JSON Schema

In [14]:
json_schema = {
 "title": "ProductInfo",
 "description": "产品信息结构，包括名称、价格与评分",
 "type": "object",
 "properties": {
  "product": {
   "type": "string",
   "description": "产品名称"
  },
  "price": {
   "type": "number",
   "description": "产品价格"
  },
  "rating": {
   "type": "number",
   "description": "评分（1-5）"
  }
 },
 "required": ["product", "price", "rating"]
}

- OpenAI function/tool schema

In [15]:
openai_tool_schema = {
  "name": "ProductInfo",
  "description": "产品信息结构，包括名称、价格与评分",
  "parameters": {
    "type": "object",
    "properties": {
      "product": {
        "type": "string",
        "description": "产品名称"
      },
      "price": {
        "type": "number",
        "description": "产品价格"
      },
      "rating": {
        "type": "number",
        "description": "评分（1-5）"
      }
    },
    "required": ["product", "price", "rating"]
  }
}

实际上，这四种格式的内容都可以直接载入，并且方法也可以进行随意进行选择：

In [16]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

structured_model1 = llm.with_structured_output(openai_tool_schema, method="json_schema")
structured_model2 = llm.with_structured_output(json_schema, method="json_mode")
structured_model3 = llm.with_structured_output(ProductInfoDict, method="function_calling")
structured_model4 = llm.with_structured_output(ProductInfo, method="json_schema")

那最后得到的结果除了 Pydantic 生成的是 `<class '__main__.ProductInfo'>` 以外，其余的都是普通的字典形态 `dict`。那这三个方法的区别在于判定的严格程度：
- json_schema：最严格，按 schema 格式生成
- function_calling：部分严格（字段完整，但内容灵活）
- json_mode：不严格，只保证 JSON 格式

这也是为什么在 LangChain 中默认使用的是 json_schema 形式。

那对于这些生成的 structured_mode 是可以通过 LCEL 链与 prompt 进行连接的，这里我们就以 Pydantic 和 json_schema 来进行演示：

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
解析以下产品评价中的信息，按格式要求输出。
评价：{input}""")

chain2 = prompt | structured_model2
chain4 = prompt | structured_model4

response1 = chain2.invoke({"input": "这款无线耳机很好用，价格399元，我给4.5分好评"})
response2 = chain4.invoke({"input": "这款无线耳机很好用，价格399元，我给4.5分好评"})
print("结构化结果1：", response1, "结构化结果2：", response2)
print(type(response1), type(response2))

结构化结果1： {'产品名称': '无线耳机', '产品优点': '很好用', '价格': '399元', '评分': '4.5分', '评价态度': '好评'} 结构化结果2： product='无线耳机' price=399.0 rating=4.5
<class 'dict'> <class '__main__.ProductInfo'>


### 3.5.2 ChatTongyi 格式化解析器

而在 ChatTongyi 中，虽然同样有 with_structured_output 方法，但是内部却并没有 method 这个参数：

![gr.ren1](images/新7.png)

但是在源代码中也是划分了两种不同的处理场景，一种是对于普通的结构化输入，那会使用 function_calling 的方式进行处理。而针对于 Pydantic 格式的内容，则会进行特殊处理将格式化的内容转为 Pydantic 对象。

#### Pydantic

In [18]:
from langchain_community.chat_models import ChatTongyi
llm = ChatTongyi(model="qwen-max") 

from pydantic import BaseModel, Field

class ProductInfo(BaseModel):
  """产品信息结构，包括名称、价格与评分"""
  product: str = Field(description="产品名称")
  price: float = Field(description="产品价格")
  rating: float = Field(description="评分（1-5的数字）")

structured_llm = llm.with_structured_output(ProductInfo)

chain = prompt | structured_llm

resp = chain.invoke({"input": "这款无线耳机很好用，价格399元，我给4.5星好评"})
print(resp)
print(type(resp))

product='无线耳机' price=399.0 rating=4.5
<class '__main__.ProductInfo'>


In [19]:
print(resp.product)   # 输出结果为“无线耳机”

无线耳机


#### TypeDict

In [20]:
from typing_extensions import TypedDict, Annotated
class ProductInfoDict(TypedDict):
  product: Annotated[str, {"description": "产品名称"}]
  price: Annotated[float, {"description": "产品价格（数字）"}]
  rating: Annotated[float, {"description": "评分（1-5）"}]
  
structured_llm = llm.with_structured_output(ProductInfoDict)

chain = prompt | structured_llm

resp = chain.invoke({"input": "这款无线耳机很好用，价格399元，我给4.5星好评"})
print(resp)
print(type(resp))

{'product': '无线耳机', 'price': 399, 'rating': 4.5}
<class 'dict'>


#### JSON Schema

In [21]:
json_schema = {
  "title": "ProductInfo",
  "description": "a product with details",
  "type": "object",
  "properties": {
    "product": {
      "type": "string",
      "description": "产品名称"
    },
    "price": {
      "type": "number",
      "description": "产品价格"
    },
    "rating": {
      "type": "number",
      "description": "评分（1-5）"
    }
  },
  "required": ["product", "price", "rating"]
}

structured_model = llm.with_structured_output(json_schema)
chain = prompt | structured_model
response = chain.invoke({"input": "这款无线耳机很好用，价格399元，我给4.5星好评"})
print("结构化结果：", response)
print(type(response))

结构化结果： {'product': '无线耳机', 'price': 399, 'rating': 4.5}
<class 'dict'>
